In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib import patches
import seaborn as sns
sns.set_theme()
import pandas as pd
import mne
from scipy.fftpack import fft
from numpy.fft import rfft, irfft, rfftfreq
from sklearn.cross_decomposition import CCA
from sklearn.decomposition import PCA
# import tensorflow as tf
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
# from meegkit import dss, ress
# from meegkit import sns as msns
# from meegkit.utils import unfold, rms, fold, tscov, matmul3d, snr_spectrum
from brainda.paradigms import SSVEP
from brainda.algorithms.utils.model_selection import (
    set_random_seeds, 
    generate_loo_indices, match_loo_indices)
from brainda.algorithms.decomposition import (
    FBTRCA, FBTDCA, FBSCCA, FBECCA, FBDSP,
    generate_filterbank, generate_cca_references)
from collections import OrderedDict
import numpy as np
from scipy.signal import sosfiltfilt
from sklearn.pipeline import clone
from sklearn.metrics import balanced_accuracy_score

In [9]:
def load_data_temp_function(eeg, meta, classes, stim_duration=5, filter=True):
    trials = meta[1:,:2] # meta organised [[hz phase time],...] meta[1:,:2] removes first row and last (third) column of meta.csv
    times = [] # gives empty arraw of times
    duration_samples = int(stim_duration*300) # because we are saying the stim_duration is 1500ms which is 1.5s

#for-loop iterates through every row of EEG data where TRG=16, appends time data at that row. 
    for index, row in eeg.loc[eeg[' TRG']==16.0].iterrows(): #? TRG: event marker or trigger signal which is last column. 16 during stim i think
        # print(index)
        # print(row)

        # checks if index>0 ensures we are not in first row because then we could do index -1
        # checks if row preceding is 0
        # takes values where the slice of the values in the next 1.5s != 18.0
        if index > 0 and eeg.iloc[index-1][' TRG'] == 0 and (not eeg.iloc[index:index+duration_samples][' TRG'].isin([18.0]).any()): 
            # print(row['time'])

            # then appends these rows to times array
            times.append(row['time'])
    times = np.array(times)
    
    # eeg['time'] = eeg['time'] - eeg['time'].iloc[0]
    # eeg = np.array([eeg.loc[eeg['time']>t].drop(columns=['time',' TRG']).to_numpy()[:1725].T for t in times])[:,:,225:]
    # eeg = np.array([eeg.loc[eeg['time']>t].drop(columns=['time']).to_numpy()[:1725].T for t in times])[:,:,225:]

    #drop the columns time, TRG, X1, X2, X3, A2
    #append resulting table of length duration_samples to resulting eeg array
    #for each place where TRG=16 after transposing
    eeg = np.array([eeg.loc[eeg['time']>t].drop(columns=['time',' TRG',' X1',' X2',' X3',' A2']).to_numpy()[:duration_samples].T for t in times])

    #performs a lower pass-band FIR filter on the EEG data
    # replaces eeg data with filtered data
    # sfreq=sampling frequency = 300hz (dsi-24)
    # l_freq=lower freq = below 5hz will be filtered out
    # h_freq= above 49hz is filtered out
    # verbose = 0 no output is printed
    # fir finite impulse response used 
    if filter:
        eeg = mne.filter.filter_data(eeg, sfreq=300, l_freq=5, h_freq=49, verbose=0, method='fir')

    # eeg = mne.filter.filter_data(eeg, sfreq=300, l_freq=5, h_freq=49, verbose=0, method='fir',phase='minimum')
        
    #classes is array of every unique row in meta
    #so eeg_temp is a 2d list with a row for every row in classes
    # just a list with 32 empty arrays
    eeg_temp = []
    for i in range(len(classes)):
        eeg_temp.append([])

    #populates the eeg_temp table with elements of eeg grouped into rows by the corresponding 'class'
    # i = index 0/1 ; freq = rows in meta
    for i,freq in enumerate(trials):
        for j,target in enumerate(classes): 
            if (freq==target).all():
                eeg_temp[j].append(eeg[i])
    
    #switches the first and second dimensions of the eeg_temp array
    eeg = np.array(eeg_temp).transpose(1,0,2,3)
    return eeg 

In [29]:
sub_dirs = ['run1/',] #'run2/','run3/','run4/', 'run5/' list of sub-directories in Simon's data for each run
duration = 1.5 # stim dur?
n_trials = 2
n_classes = 32
n_channels = 19

# load experimental dataset
# eeg_whole is a 4D array where first dimension is n_trials * # of runs
# each entry in the second class represents a different unique clas (unique rows in meta) second dimension
# thrid dimension represents the specific channel, which there are 19 of in this case because of hte DSI-24
eeg_whole = np.zeros((n_trials*len(sub_dirs),n_classes,n_channels,int(duration*300)))

# target_tab stores the classes and their corresponding index in a dictionary
target_tab = {}

# i_class_ = 0
for i_dir,sub_dir in enumerate(sub_dirs): #cycles through each run (1-5 in this case) index(0-4 obvs)
    print(i_dir) # index of each run

    # read the eeg and meta data
    data_path = "../data/eeg_recordings/32-class_speller/DSI-24/Simon/" + sub_dir
    eeg = pd.read_csv(data_path + 'eeg.csv').astype(float)
    meta = np.loadtxt(data_path + 'meta.csv', delimiter=',', dtype=float) 
    trials = meta[1:,:2] 
    classes = np.unique(trials, axis=0)
    more_targets = {tuple(target):index for index,target in enumerate(classes)}

    # add current directory's classes with indexes to target_tab dictionary
    target_tab.update(more_targets)

    # calls load function from block 2 and populates zeros of eeg_whole
    eeg = load_data_temp_function(eeg, meta, classes, stim_duration=duration,filter=False)

    # inserts run's eeg data into the eeg_whole array
    eeg_whole[i_dir*n_trials:(i_dir+1)*n_trials,:,:,:] = eeg
    print(eeg)
    # i_class+=3

eeg = eeg_whole
# target by trial stores classes, but repeated for each trial.
target_by_trial = [list(target_tab.keys())] * n_trials*len(sub_dirs)
eeg.shape, np.array(target_by_trial).shape 

0
[183782.5098731 183786.5621062 183790.5553075 183794.5664193
 183798.5879069 183802.6318414 183808.5400644 183812.5234152
 183816.5300285 183820.5578087 183824.5733585 183828.5912418
 183832.5847699 183836.6273905 183840.6103898 183844.6149571
 183848.6303273 183852.6129276 183856.6193113 183860.6300076
 183864.6547343 183868.6406518 183872.6544112 183876.6779566
 183880.7142016 183884.7211261 183888.7510152 183892.7683116
 183896.7548806 183900.7450112 183904.7323744 183908.7481458
 183912.7741406 183916.8004511 183920.7776201 183924.7681248
 183928.7981347 183932.8110515 183936.8041671 183940.7910808
 183944.7907793 183948.8108872 183952.8195059 183956.8706547
 183962.9639435 183966.980414  183970.9721776 183974.9863849
 183979.0124395 183983.0138801 183987.0170074 183991.0138388
 183995.0396563 183999.0222048 184003.0331674 184007.062417
 184011.0532916 184015.0750537 184019.0897029 184023.0799826
 184027.0806632 184031.1131013 184035.1093627 184039.0959326]
[[[[   0.28481922    1

((2, 32, 19, 450), (2, 32, 2))

In [5]:
n_trials = eeg.shape[0]
# classes = [0,7,15,23,31]
classes = range(32) 
n_classes = len(classes) # = int 32 classes 
y = np.array([list(target_tab.values())] * n_trials).T.reshape(-1)
eeg_temp = eeg[:n_trials,classes,:,42:]
# X = eeg.swapaxes(0,1).reshape(-1,*eeg.shape[2:])
X = eeg_temp.swapaxes(0,1).reshape(-1,*eeg_temp.shape[2:])

# extracts frequency targets from target_by_trial, picks first row of array : selects all columns in that row next 0 selects first element of each row in that slice
freq_targets = np.array(target_by_trial)[0,:,0]
# same thing but with phases and picks second element
phase_targets = np.array(target_by_trial)[0,:,1]
# harmonics are integer multiples of fundamental frequency. here 5 harmonics are used to generate cca
n_harmonics = 5
# number of frequency bands used to make filter banks for the eeg data
n_bands = 3
# sampling rate of 300hz on dsi24
srate = 300
# duration of each trial or segment of eeg data in seconds.
duration = 1

# stores the reference signals for cca. each reference has several harmonics which can be compared to the eeg signal
Yf = generate_cca_references(
    freq_targets, srate, duration, 
    phases=phase_targets, 
    n_harmonics=n_harmonics)

# 
wp = [[8*i, 90] for i in range(1, n_bands+1)]
ws = [[8*i-2, 95] for i in range(1, n_bands+1)]
filterbank = generate_filterbank(
    wp, ws, srate, order=4, rp=1)
filterweights = np.arange(1, len(filterbank)+1)**(-1.25) + 0.25
set_random_seeds(64)
l = 5
models = OrderedDict([
    # ('fbscca', FBSCCA(
    #         filterbank, filterweights=filterweights)),
    # ('fbecca', FBECCA(
    #         filterbank, filterweights=filterweights)),
    ('fbdsp', FBDSP(
            filterbank, filterweights=filterweights)),
    ('fbtrca', FBTRCA(
            filterbank, filterweights=filterweights)),
    ('fbtdca', FBTDCA(
            filterbank, l, n_components=8, 
            filterweights=filterweights)),
])
events = []
# for j_class in range(32):
for j_class in classes:
    events.extend([str(target_by_trial[i_trial][j_class]) for i_trial in range(n_trials)])
events = np.array(events)
subjects = ['1'] * (n_classes*n_trials)
meta = pd.DataFrame(data=np.array([subjects,events]).T, columns=["subject", "event"])
set_random_seeds(42)
loo_indices = generate_loo_indices(meta)
best_model = None
best_score = -1

for model_name in models:
    if model_name == 'fbtdca':
        filterX, filterY = np.copy(X[..., :int(srate*duration)+l]), np.copy(y)
    else:
        filterX, filterY = np.copy(X[..., :int(srate*duration)]), np.copy(y)
    
    filterX = filterX - np.mean(filterX, axis=-1, keepdims=True)

    n_loo = len(loo_indices['1'][events[0]])
    loo_accs = []
    for k in range(n_loo):
        train_ind, validate_ind, test_ind = match_loo_indices(
            k, meta, loo_indices)
        train_ind = np.concatenate([train_ind, validate_ind])

        trainX, trainY = filterX[train_ind], filterY[train_ind]
        testX, testY = filterX[test_ind], filterY[test_ind]

        model = clone(models[model_name]).fit(
            trainX, trainY,
            Yf=Yf
        )
        pred_labels = model.predict(testX)
        score = balanced_accuracy_score(testY, pred_labels)
        loo_accs.append(score)
        if score > best_score:
            best_score = score
            best_model = model
    print("Model:{} LOO Acc:{:.2f}".format(model_name, np.mean(loo_accs)))

AttributeError: 'FBDSP' object has no attribute 'n_components'

In [5]:
print(loo_accs)
print(n_loo)

[0.96875, 0.9375, 0.875, 0.9375, 0.84375, 0.875, 0.78125, 0.96875, 0.875, 0.96875]
10


In [6]:
import pickle
with open("../reports/trained_models/32-class_speller/DSI-24/Aidan/fbtdca_1s.pkl", 'wb') as filehandler:
    pickle.dump(best_model, filehandler)

In [7]:
filterX.shape, X.shape, eeg_temp.shape

((320, 19, 305), (320, 19, 408), (10, 32, 19, 408))